In [ ]:

import csv
import random

class Warehouse:
  def __init__(self):
    self.inventory = []
    self.inventory_weights= []
    self.batches = []
    #TAMANO DE LOS ITEMS
    self.weights= [10,14,20,15,20,12,16,16,20,25,19,16,15]
    #------------------------------------------------------------------------#
    self.sizes = [[3,4,3], [3,6,3], [4,8,4], [3,7,3], [4,8,4], [4,4,3], [5,4,4], [4,6,3], [6,13,6], [7,15,7], [4,11,4], [3,11,3], [3,10,3]]

  def generate_inventory(self, num):
    for i in range(num):
      x = random.randint(0,12)
      self.inventory.append(self.sizes[x])
      self.inventory_weights.append(self.weights[x])
    
    with open('inventory.csv', 'w', newline='') as file:
      wr = csv.writer(file, quoting=csv.QUOTE_ALL)
      wr.writerows(self.inventory)
    
  def load_inventory(self):
    with open('inventory.csv', newline='') as file:
      reader = csv.reader(file)
      data = list(reader)
      tmp = []
      for d in data:
        tmp.append([int(i) for i in d])
      self.inventory = tmp 

  def create_batch(self, container_x, container_y, container_z, n_items,):
    #------------------------------------------------------------------------#
    batch = Batch(container_x, container_y, container_z, self.inventory[:n_items],self.inventory_weights[:n_items])
    del self.inventory[:n_items]
    #------------------------------------------------------------------------#
    del self.inventory_weights[:n_items]
    #------------------------------------------------------------------------#
    batch.pack_batch()
    self.batches.append(batch)
  
  def print_batches(self):
    for batch in self.batches:
      print('Para la entrega: ' + str(self.batches.index(batch)+1))
      batch.print_batch()
      #------------------------------------------------------------------------#
      print("\nRedistribuyendo los item en los contenedores\n")
      batch.refill_container()

  def plot_batches_1(self):
    contenedores = []
    cantidades = []
    times = []
    for b in self.batches:
      contenedores.append(len(b.containers))
      cantidades.append(b.batch_items_cant)
      times.append(b.time)

    plt.title('Tamano de contenedor fijo - ' + str(b.x_cells) + ',' + str(b.skyline-1) + ',' + str(b.z_cells))
    plt.xlabel('Cantidad de items')
    plt.ylabel('Cantidad de contenedores')
    plt.plot(cantidades,contenedores)
    plt.show()

    plt.title('Tiempo de ejecucion')
    plt.xlabel('Cantidad de items')
    plt.ylabel('Tiempo')
    plt.plot(cantidades, times)
    plt.show()

In [ ]:
import time
import random
import numpy as np

class Batch:
  def __init__(self, container_x, container_y, container_z, batch_items, weights):
    self.x_cells = container_x
    self.skyline = container_y+1
    self.z_cells = container_z
    self.batch_items = batch_items
    self.empty_container = [0]*(container_x*container_z)
    self.containers = []
    self.DP = dict()
    self.leftover = 0
    self.time = 0
    self.batch_items_cant = len(batch_items)
    
    #------------------------------------------------------------------------#
    self.weights=weights
    self.ideal_weight=0
    #------------------------------------------------------------------------#
      
  def print_batch(self):
    cont = 1
    total = 0
    total_weight=0
    print("\n")
    for container in self.containers:
      total+= int(str(container))
      print('Caja #' + str(cont) + ' tiene ' + str(container) + ' items')
      cont += 1
    self.leftover= self.batch_items_cant-total
    print('Quedaron ' + str(self.leftover) + ' items por guardar en el lote')
    
    #------------------------------------------------------------------------#
    print('Total Items:' + str(total))
    for w in self.weights:
      total_weight+= w

    self.ideal_weight= total_weight/(len(self.containers))
    print('Peso Total: ' + str(total_weight) )
    print('Peso Ideal: ' + (str(self.ideal_weight)))
    #------------------------------------------------------------------------#

  #------------------------------------------------------------------------#
  def refill_container(self):
    containers_cant= len(self.containers)
    cont=0
    extra_w=0
    weights_aux= self.weights
    self.containers=[]
    while cont < containers_cant:
      items_to_pack=[]
      items_cont=0
      container_actual_weight=0
      for item in weights_aux:
        extra_w = weights_aux[items_cont]
        container_actual_weight += weights_aux[items_cont]
        if item != 0 and items_cont <= len(self.weights): 
          if container_actual_weight <= self.ideal_weight:
            items_to_pack.append(item)
            weights_aux[items_cont] = 0
            items_cont += 1
          else:
            container_actual_weight-= extra_w            
            print('Peso Caja #'+str(cont+1)+': ' +str(container_actual_weight))
            self.containers.append(len(items_to_pack))
            items_to_pack=[]
            total_aux=0
            break;
        else:
          items_cont+=1
      cont+=1    
    
    cont = 1
    total = 0
    total_weight=0
    print("\n")
    for container in self.containers:
      total+= int(str(container))
      print('Caja #' + str(cont) + ' tiene ' + str(container) + ' items')
      cont += 1
    self.leftover= self.batch_items_cant-total
    print('Quedaron ' + str(self.leftover) + ' items por guardar en el lote')
  #------------------------------------------------------------------------#

  def getPos(self, a , b):
    return ((a*self.z_cells)+b)

  def pack_batch(self):
    t = time.clock()
    packed = 0
    while self.batch_items:
      self.DP = dict()
      items_packed = []
      temp = self.batch_items[:]
      cant = self.fill_container(packed, self.empty_container[:], self.batch_items, items_packed)
      for item in items_packed:
        i = temp.index(item)
        del temp[i]
      self.batch_items = temp
      self.containers.append(cant)
    self.time = time.clock() - t

  def fill_container(self, i, h, items, it_packed):
    h_tuple = tuple(h)

    if (i,h_tuple) in self.DP :
      return self.DP[i,h_tuple]

    if (max(h)>= self.skyline):
      return 0
    
    if items:
      ix, iy, iz = tuple(items.pop())

      for t in range(2):
        if t == 1:
          ix, iz = iz, ix

        for u in range(self.x_cells-(ix-1)):
          for j in range(self.z_cells-(iz-1)):
            new_h = h[:]
            p = self.getPos(u,j)
            local_hmax = new_h[p]

            for k in range(ix):
              for l in range(iz):
                kl = self.getPos(u+k, j+l)
                local_hmax = max(local_hmax, new_h[kl])

            if (local_hmax + iy < self.skyline):
              for k in range(ix):
                for l in range(iz):
                  kl = self.getPos(u+k, j+l)
                  new_h[kl] = local_hmax + iy

              if (i,h_tuple) in self.DP:
                self.DP[i,h_tuple] = max(self.DP[i,h_tuple], self.fill_container(i+1, new_h[:], items, it_packed) +1)
              else:
                self.DP[i,h_tuple] = self.fill_container(i+1, new_h[:], items, it_packed) +1
                if t == 1:
                  it_packed.append([iz,iy,ix])
                else:
                  it_packed.append([ix,iy,iz])

    if (i,h_tuple) in self.DP :
      return self.DP[i,h_tuple]
    else:
      return 0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1

warehouse = Warehouse()
warehouse.generate_inventory(500)

warehouse.load_inventory()
warehouse.create_batch(20,30,25,500)
warehouse.print_batches()


bs = warehouse.batches
contenedores = []
cantidades = []
times = []
for b in bs:
  contenedores.append(len(b.containers))
  cantidades.append(b.batch_items_cant)
  times.append(b.time)



Para la entrega: 1


Caja #1 tiene 53 items
Caja #2 tiene 52 items
Caja #3 tiene 38 items
Caja #4 tiene 39 items
Caja #5 tiene 36 items
Caja #6 tiene 32 items
Caja #7 tiene 27 items
Caja #8 tiene 4 items
Quedaron 219 items por guardar en el lote
Total Items:281
Peso Total: 8512
Peso Ideal: 1064.0

Redistribuyendo los item en los contenedores

Peso Caja #1: 1061
Peso Caja #2: 1052
Peso Caja #3: 1059
Peso Caja #4: 1062
Peso Caja #5: 1048
Peso Caja #6: 1054
Peso Caja #7: 1061
Peso Caja #8: 1059


Caja #1 tiene 61 items
Caja #2 tiene 63 items
Caja #3 tiene 62 items
Caja #4 tiene 62 items
Caja #5 tiene 63 items
Caja #6 tiene 64 items
Caja #7 tiene 61 items
Caja #8 tiene 61 items
Quedaron 3 items por guardar en el lote
